# 📐 Pi_AutoRunner — CALCULATION_OF_NUMBER_PI

Dieses Notebook:
1) installiert System-Pakete (inkl. CUDA Toolkit),
2) klont `error-wtf/CALCULATION_OF_NUMBER_PI`,
3) erkennt automatisch einen Python-Entry-Point und startet ihn (konfigurierbar),
4) **installiert die im Repo bereits enthaltene `.deb`** `pi-error-wtf-v1.0-Linux.deb` (kein Neubau),
5) hat einen RAM/Disk-Heuristik-Check für sichere **Max-Digits**.

> Tipp: Wenn sich die Struktur ändert, setze unten `ENTRYPOINT` manuell.

In [ ]:
# === 0) System dependencies ===
!sudo apt-get update -y
!sudo apt-get install -y build-essential cmake libboost-dev nvidia-cuda-toolkit dpkg-dev rsync git
!python3 -V
!gcc --version | head -n1 || true
!nproc --all || true
!free -h || true
!df -h / || true
print("CUDA (nvcc) present:")
!nvcc --version || true

In [ ]:
# === 1) Clone repository ===
REPO_URL = "https://github.com/error-wtf/CALCULATION_OF_NUMBER_PI.git"
REPO_DIR = "/content/CALCULATION_OF_NUMBER_PI"

import os, shutil, subprocess
if os.path.exists(REPO_DIR):
    shutil.rmtree(REPO_DIR)
!
git clone --depth=1 "$REPO_URL" "$REPO_DIR"
!find "$REPO_DIR" -maxdepth 2 -type f -printf "%p\n" | sort

## Parameter — Entry-Point & Digits
Du kannst hier Entry-Point und gewünschte Ziffern setzen. Wenn leer, wird automatisch gesucht.

In [ ]:
#@title Runner-Einstellungen
ENTRYPOINT = ""  #@param {type:"string"}
ENTRY_ARGS_TEXT = ""  #@param {type:"string"}
RUN_SOURCE = True  #@param {type:"boolean"}
INSTALL_DEB = True  #@param {type:"boolean"}

# ENTRY_ARGS_TEXT wird in eine Liste gesplittet (shell-like)
import shlex
ENTRY_ARGS = shlex.split(ENTRY_ARGS_TEXT) if ENTRY_ARGS_TEXT else []
print({"ENTRYPOINT": ENTRYPOINT, "ENTRY_ARGS": ENTRY_ARGS, "RUN_SOURCE": RUN_SOURCE, "INSTALL_DEB": INSTALL_DEB})

## Heuristik: sichere Max-Digits (RAM & Disk)
Sehr konservative Schätzung (anpassbar).

In [ ]:
import shutil

def mem_free_bytes():
    try:
        with open("/proc/meminfo") as f:
            for line in f:
                if line.startswith("MemAvailable:"):
                    return int(line.split()[1]) * 1024
    except Exception:
        return 1_000_000_000

def disk_free_bytes(path="/content"):
    return shutil.disk_usage(path).free

# konservativ; je nach Implementierung anpassen
BYTES_PER_DIGIT_RAM = 32   # Arbeits-Buffer
BYTES_PER_DIGIT_DISK = 2   # Ausgabe + Temp

ram = mem_free_bytes()
disk = disk_free_bytes()
max_ram = ram // BYTES_PER_DIGIT_RAM
max_disk = disk // BYTES_PER_DIGIT_DISK
safe = int(min(max_ram, max_disk) * 0.7)
print(f"Free RAM  : {ram/1e9:.2f} GB")
print(f"Free Disk : {disk/1e9:.2f} GB")
print(f"RAM-bound  : {max_ram:,} digits (est)")
print(f"Disk-bound : {max_disk:,} digits (est)")
print(f"Suggested  : {safe:,} digits")

## Source-Runner (auto-detect `main.py` / `runner.py` / `run.py`)
Wenn `ENTRYPOINT` oben leer ist, wird automatisch gesucht.

In [ ]:
import os, glob, subprocess, sys, shlex
os.chdir(REPO_DIR)

if RUN_SOURCE:
    ep = ENTRYPOINT.strip() if ENTRYPOINT else ""
    if not ep:
        candidates = ["main.py", "runner.py", "run.py"]
        # Fallback: größte top-level .py
        for p in sorted(glob.glob("*.py"), key=lambda p: os.path.getsize(p), reverse=True):
            if p not in candidates:
                candidates.append(p)
        for c in candidates:
            if os.path.exists(c):
                ep = c
                break
    print("Detected ENTRYPOINT:", ep or "<none>")
    if ep:
        cmd = ["python3", "-u", ep] + ENTRY_ARGS
        print("Running:", " ".join(shlex.quote(x) for x in cmd))
        proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for line in proc.stdout:
            sys.stdout.write(line)
        proc.wait()
        print("\n-- exit code:", proc.returncode)
    else:
        print("No entrypoint found. Set ENTRYPOINT manually in the params cell.")
else:
    print("RUN_SOURCE=False -> übersprungen.")

## `.deb` aus dem Repo installieren (kein Neubau)
Sucht nach `pi-error-wtf-v1.0-Linux.deb` und installiert diese direkt mit `dpkg -i`.

In [ ]:
import os, glob
DEB_NAME = "pi-error-wtf-v1.0-Linux.deb"

if INSTALL_DEB:
    debs = [p for p in glob.glob(os.path.join(REPO_DIR, "**", "*.deb"), recursive=True)
            if os.path.basename(p) == DEB_NAME]
    if not debs:
        print(f"❗ {DEB_NAME} nicht gefunden. Prüfe Pfad/Dateiname im Repo.")
    else:
        deb_path = debs[0]
        print("Gefunden:", deb_path)
        !chmod a+x "$deb_path"
        !sudo dpkg -i "$deb_path" || true
        print("\nWenn Abhängigkeiten fehlen: bitte ausführen ->  sudo apt-get -f install -y")
else:
    print("INSTALL_DEB=False -> übersprungen.")

## (Optional) Installation prüfen & mögliche Launcher finden

In [ ]:
!which -a pi-calc || true
!ls -l /usr/local/bin || true
!ls -l /opt || true